# Building Agents 
 
> Note: Optionally, see [these slides](https://docs.google.com/presentation/d/13c0L1CQWAL7fuCXakOqjkvoodfynPJI4Hw_4H76okVU/edit?usp=sharing) and [langgraph_101.ipynb](langgraph_101.ipynb) for context before diving into this notebook!

We're going to build an email assistant from scratch, starting here with

1) the agent architecture (using [LangGraph JS](https://langchain-ai.github.io/langgraphjs/)) and following with
2) testing (using [LangSmith](https://docs.smith.langchain.com/)),
3) human-in-the-loop, and
4) memory. This diagram show how these pieces will fit together:

![overview-img](img/overview.png)

### Tool Definition

Let's start by defining some simple tools that an email assistant will use with [`tool`](https://v03.api.js.langchain.com/functions/_langchain_core.tools.tool-1.html)s:

In [1]:
import { z } from "zod";
import { tool } from "@langchain/core/tools";

// Define schema for email writing
const writeEmailSchema = z.object({
  to: z.string().describe("Email address of the recipient"),
  subject: z.string().describe("Subject line for the email"),
  content: z.string().describe("Main body text of the email")
});

// Write email tool
const writeEmail = tool(
  async (input: z.infer<typeof writeEmailSchema>) => {
  const { to, subject, content } = input;
  // Placeholder response - in real app would send email
  return `Email sent to ${to} with subject '${subject}' and content: ${content}`;
}, {
  name: "write_email",
  description: "Write and send an email.",
  schema: writeEmailSchema,
});

// Define schema for scheduling meeting
const scheduleMeetingSchema = z.object({
  attendees: z.array(z.string()).describe("List of attendees' emails"),
  subject: z.string().describe("Meeting title or subject"),
  duration_minutes: z.number().describe("Duration of meeting in minutes"),
  preferred_day: z.string().describe("Preferred date for the meeting (e.g., 'tomorrow', '2024-07-28')"),
  start_time: z.string().describe("Start time of the meeting (e.g., '2:00 PM', '14:00')")
});

// Schedule meeting tool
const scheduleMeeting = tool(
  async (input: z.infer<typeof scheduleMeetingSchema>) => {
  const { attendees, subject, duration_minutes, preferred_day, start_time } = input;
  // Placeholder response - in real app would check calendar and schedule
  // Basic date parsing, consider a robust library for production
  let dateStr = preferred_day;
  try {
      // Attempt to format if it's a parsable date string
      const parsedDate = new Date(preferred_day);
      if (!isNaN(parsedDate.getTime())) {
           dateStr = parsedDate.toLocaleDateString('en-US', { weekday: 'long', month: 'long', day: 'numeric', year: 'numeric' });
      }
      // If preferred_day is not a valid date string (e.g. "tomorrow"), use it as is.
  } catch (e) {
      // Fallback to using preferred_day as is if parsing fails
  }
  return `Meeting '${subject}' scheduled on ${dateStr} at ${start_time} for ${duration_minutes} minutes with ${attendees.join(', ')}`;
},{
  name: "schedule_meeting",
  description: "Schedule a calendar meeting.",
  schema: scheduleMeetingSchema,
  
});

// Define schema for checking calendar
const checkCalendarSchema = z.object({
  day: z.string().describe("Day to check calendar availability (e.g., 'tomorrow', 'next Monday', '2024-07-28')")
});

// Check calendar availability tool
const checkCalendarAvailability = tool(async (input: z.infer<typeof checkCalendarSchema>) => {
  const { day } = input;
  // Placeholder response - in real app would check actual calendar
  return `Available times on ${day}: 9:00 AM, 2:00 PM, 4:00 PM`;
}, {
  name: "check_calendar_availability",
  description: "Check calendar availability for a given day.",
  schema: checkCalendarSchema,
});


// Define schema for Done tool
const doneSchema = z.object({
  // The LLM will decide if the task is 'done' and pass true/false.
  done: z.boolean().describe("Task completion status. Set to true when the current task or series of actions is complete.")
});

const doneTool = tool(async (input: z.infer<typeof doneSchema>) => {
  // The observation returned to the LLM.
    // The observation returned to the LLM.
    // The 'input.done' (true/false from LLM) is used here to make the observation more specific.
    return input.done ? "Task marked as completed successfully by LLM." : "Task completion not explicitly confirmed as true by LLM, but Done tool was called.";
  }, {
  name: "Done",
  description: "Signals that the current e-mail related task or sequence of tool calls is complete. Call this when no more actions are needed for the current e-mail.",
  schema: doneSchema,
});

### Augmenting the LLM with Tools

Now we connect these tools to a LLM. We'll use LangChain's [`initChatModel`](https://js.langchain.com/docs/how_to/chat_models_universal_init/) interface, which allows us to initialize many different chat models.

We [enforce tool use](https://js.langchain.com/docs/how_to/tool_choice/) by setting `tool_choice: "required"` (or the equivalent parameter for the specific model integration, like in `bindTools`).

In [2]:
import { initChatModel } from "langchain/chat_models/universal";
import { AIMessage } from "@langchain/core/messages";

// Declare 'output' in a scope accessible by subsequent cells
let output: AIMessage | null = null;

const llm = await initChatModel(
  "openai:gpt-4o",
  { temperature: 0.0 }
);

const llmWithTools = llm.bindTools([writeEmail]); 

const emailInput = {
  author: "System Admin <sysadmin@company.com>",
  to: "Development Team <dev@company.com>",
  subject: "Scheduled maintenance - database downtime",
  email_thread: "Hi team,\n\nThis is a reminder that we'll be performing scheduled maintenance on the production database tonight from 2AM to 4AM EST. During this time, all database services will be unavailable.\n\nPlease plan your work accordingly and ensure no critical deployments are scheduled during this window.\n\nThanks,\nSystem Admin Team"
};

const messages = [
  {
    role: "user",
    content: "Call the 'write_email' tool in order to respond to this email: " + JSON.stringify(emailInput, null, 2)
  }
];

output = await llmWithTools.invoke(messages);

console.log("LLM Output from Cell 2:", output);
if (output && output.tool_calls) {
  console.log("Tool Calls from Cell 2:", output.tool_calls);
} else {
  console.log("No tool calls found in the output from Cell 2.");
}

LLM Output from Cell 2: AIMessage {
  "id": "chatcmpl-BWXe4SzWPq7nwxgrNqQYsxwtqGxUG",
  "content": "",
  "additional_kwargs": {
    "tool_calls": [
      {
        "id": "call_I1389E1FFGgbeNAkmdJriJyG",
        "type": "function",
        "function": "[Object]"
      }
    ]
  },
  "response_metadata": {
    "tokenUsage": {
      "promptTokens": 200,
      "completionTokens": 73,
      "totalTokens": 273
    },
    "finish_reason": "tool_calls",
    "model_name": "gpt-4o-2024-08-06",
    "usage": {
      "prompt_tokens": 200,
      "completion_tokens": 73,
      "total_tokens": 273,
      "prompt_tokens_details": {
        "cached_tokens": 0,
        "audio_tokens": 0
      },
      "completion_tokens_details": {
        "reasoning_tokens": 0,
        "audio_tokens": 0,
        "accepted_prediction_tokens": 0,
        "rejected_prediction_tokens": 0
      }
    },
    "system_fingerprint": "fp_f5bdcc3276"
  },
  "tool_calls": [
    {
      "name": "write_email",
      "args": {
       

In [3]:
console.log(output.tool_calls);

[
  {
    name: 'write_email',
    args: {
      to: 'sysadmin@company.com',
      subject: 'Re: Scheduled maintenance - database downtime',
      content: 'Hi System Admin Team,\n' +
        '\n' +
        'Thank you for the reminder. We will ensure that no critical deployments are scheduled during the maintenance window and will plan our work accordingly.\n' +
        '\n' +
        'Best regards,\n' +
        'Development Team'
    },
    type: 'tool_call',
    id: 'call_I1389E1FFGgbeNAkmdJriJyG'
  }
]


In [4]:
const args = output.tool_calls?.[0]?.args;

In [5]:
const writeEmailResult = await writeEmail.invoke(args);
console.log(writeEmailResult);

Email sent to sysadmin@company.com with subject 'Re: Scheduled maintenance - database downtime' and content: Hi System Admin Team,

Thank you for the reminder. We will ensure that no critical deployments are scheduled during the maintenance window and will plan our work accordingly.

Best regards,
Development Team


## Building our email assistant

We'll combine a [router and agent](https://langchain-ai.github.io/langgraphjs/tutorials/workflows/) to build our email assistant.

![agent_workflow_img](img/email_workflow.png)

### Router

The routing step handles the triage decision. 

The triage router only focuses on the triage decision, while the agent focuses *only* on the response. 

When building an agent, it's important to consider the information that you want to track over time. LangGraph uses a state object to manage this. For chat history, a common approach is to use a `messages` key in the state, often managed by a reducer like the one provided by `MessagesAnnotation` (which internally uses a prebuilt reducer called `messagesStateReducer`). This reducer appends new messages to the history and can handle special values that e.g. delete specific messages.

This allows for all standard message types to be part of the state. Additionally, LangGraph gives you flexibility to track other information beyond messages. We'll define a custom `State` object (typically using Zod for schema definition in TypeScript) that includes a `messages` key for our chat history (allowing for standard message types) and adds a `classification_decision` key:

In [7]:
import type { BaseMessage } from "@langchain/core/messages";
import { addMessages, Messages } from "@langchain/langgraph";

// Allow extension of Zod for LangGraph
import "@langchain/langgraph/zod";

const StateSchema = 
  z.object({
    messages: z
      .custom<BaseMessage[]>()
      .default(() => [])
      .langgraph.reducer<Messages>((left, right) => addMessages(left, right)),
    email_input: z.record(z.any()),
    classification_decision: z.enum(["ignore", "respond", "notify"])
  });
// Add this right after your StateSchema definition (cell 17)
type State = z.infer<typeof StateSchema>;

#### Triage node

We define a TypeScript function with our triage routing logic.

For this, we use [structured outputs](https://js.langchain.com/docs/concepts/structured_outputs/) with a Zod schema. Zod schemas are particularly useful for defining structured output schemas in TypeScript because they offer type hints and validation. The descriptions in the Zod schema are important because they get passed as part of the JSON schema to the LLM to inform the output coercion (often using the `.withStructuredOutput()` method).

In [8]:

import { END } from "@langchain/langgraph";
import { initChatModel } from "langchain/chat_models/universal";
import { Runnable } from "@langchain/core/runnables";
import { HumanMessage } from "@langchain/core/messages";

// PROMPTS 

const defaultBackground = `
I'm Lance, a software engineer at LangChain.
`;

const defaultTriageInstructions = `
Emails that are not worth responding to:
- Marketing newsletters and promotional emails
- Spam or suspicious emails
- CC'd on FYI threads with no direct questions

There are also other things that should be known about, but don't require an email response. For these, you should notify (using the \`notify\` response). Examples of this include:
- Team member out sick or on vacation
- Build system notifications or deployments
- Project status updates without action items
- Important company announcements
- FYI emails that contain relevant information for current projects
- HR Department deadline reminders
- Subscription status / renewal reminders
- GitHub notifications

Emails that are worth responding to:
- Direct questions from team members requiring expertise
- Meeting requests requiring confirmation
- Critical bug reports related to team's projects
- Requests from management requiring acknowledgment
- Client inquiries about project status or features
- Technical questions about documentation, code, or APIs (especially questions about missing endpoints or features)
- Personal reminders related to family (wife / daughter)
- Personal reminder related to self-care (doctor appointments, etc)
`;

// Agentic workflow triage user prompt
const triageUserPrompt = `
Please determine how to handle the below email thread:

From: {author}
To: {to}
Subject: {subject}
{email_thread}`;

// Define the router schema using Zod
const RouterSchema = z.object({
  reasoning: z.string().describe("Step-by-step reasoning behind the classification."),
  classification: z.enum(["ignore", "respond", "notify"]).describe(
    "The classification of an email: 'ignore' for irrelevant emails, " +
    "'notify' for important information that doesn't need a response, " +
    "'respond' for emails that need a reply"
  )
});

  // Initialize the LLM
  const llm = await initChatModel("openai:gpt-4o", {
    temperature: 0.0,
  });

  type EmailData = {
    id: string;
    thread_id: string;
    from_email: string;
    subject: string;
    page_content: string;
    send_time: string;
    to_email: string;
  };

function parseEmail(email: EmailData): {
  author: string;
  to: string;
  subject: string;
  emailThread: string;
} {
  try {
    // Extract key information from email data
    const author = email.from_email;
    const to = email.to_email;
    const subject = email.subject;
    const emailThread = email.page_content;

    return { author, to, subject, emailThread };
  } catch (error) {
    console.error("Error parsing email:", error);
    throw new Error("Failed to parse email");
  }
}

function formatEmailMarkdown(
  
  subject: string,
  author: string,
  to: string,
  emailThread: string,
): string {
  return `## Email: ${subject}

**From**: ${author}
**To**: ${to}

${emailThread}`;
}

const llmRouter: Runnable = llm.withStructuredOutput(RouterSchema);

// Triage router function 
async function triageRouter(state) {
  // Parse email components
  const { author, to, subject, emailThread } = parseEmail(state.email_input);
  
  // Format prompts
  const systemPrompt = triageSystemPrompt
    .replace("{background}", defaultBackground)
    .replace("{triage_instructions}", defaultTriageInstructions);

  const userPrompt = triageUserPrompt
    .replace("{author}", author)
    .replace("{to}", to)
    .replace("{subject}", subject)
    .replace("{email_thread}", emailThread);

  // Invoke the LLM with structured output
  const result = await llmRouter.invoke([
    { role: "system", content: systemPrompt },
    { role: "user", content: userPrompt }
  ]);

  if (result.classification === "respond") {
    console.log("📧 Classification: RESPOND - This email requires a response");
    return {
      goto: "response_agent",
      update: {
        messages: [
          new HumanMessage({
            content: `Respond to the email: \n\n${formatEmailMarkdown(subject, author, to, emailThread)}`
          })
        ],
        classification_decision: result.classification
      }
    };


  } else if (result.classification === "ignore") {
    console.log("🚫 Classification: IGNORE - This email can be safely ignored");
    return {
      goto: END,
      update: {
        classification_decision: result.classification
      }
    };
  } else if (result.classification === "notify") {
    console.log("🔔 Classification: NOTIFY - This email contains important information");
    // For now, we go to END. But we will add to this later!
    return {
      goto: END,
      update: {
        classification_decision: result.classification
      }
    };
  } else {
    throw new Error(`Invalid classification: ${result.classification}`);
  }
}

117:24 - Cannot find name 'triageSystemPrompt'. Did you mean 'systemPrompt'?


We use concepts similar to [Commands (how to update state and jump to nodes)](https://langchain-ai.github.io/langgraphjs/how-tos/command/) in LangGraph to both update the state and select the next node to visit. This is a useful alternative to edges.

### Agent

Now, let's build the agent.

#### LLM node

Here, we define the LLM decision-making node. This node takes in the current state, calls the LLM, and updates `messages` with the LLM output. 

In [80]:
console.log(AGENT_TOOLS_PROMPT);


1. write_email(to, subject, content) - Send emails to specified recipients
2. schedule_meeting(attendees, subject, duration_minutes, preferred_day, start_time) - Schedule calendar meetings where preferred_day is a datetime object
3. check_calendar_availability(day) - Check available time slots for a given day
4. Done - E-mail has been sent



In [81]:
console.log(agentSystemPrompt);


<Role>
You are a top-notch executive assistant who cares about helping your executive perform as well as possible.
</Role>

<Tools>
You have access to the following tools to help manage communications and schedule:
{tools_prompt}
</Tools>

<Instructions>
When handling emails, follow these steps:
1. Carefully analyze the email content and purpose
2. IMPORTANT --- always call a tool and call one tool at a time until the task is complete: 
3. For responding to the email, draft a response email with the write_email tool
4. For meeting requests, use the check_calendar_availability tool to find open time slots
5. To schedule a meeting, use the schedule_meeting tool with a datetime object for the preferred_day parameter
   - Today's date is 2025-05-11 - use this for scheduling meetings accurately
6. If you scheduled a meeting, then draft a short response email using the write_email tool
7. After using the write_email tool, the task is complete
8. If you have sent the email, then use the Done

In [ ]:
// Collect all tools
const tools = [writeEmail, scheduleMeeting, checkCalendarAvailability, Done];
const toolsByName = new Map(tools.map(tool => [tool.name, tool]));


// Initialize the LLM, enforcing tool use
 const llm = await initChatModel("openai:gpt-4o", {
  temperature: 0.0,
});

const llmWithTools = llm.bindTools(tools, { toolChoice: "required" });

const agentSystemPrompt = `
<Role>
You are a top-notch executive assistant who cares about helping your executive perform as well as possible.
</Role>

<Tools>
You have access to the following tools to help manage communications and schedule:
{tools_prompt}
</Tools>

<Instructions>
When handling emails, follow these steps:
1. Carefully analyze the email content and purpose
2. IMPORTANT --- always call a tool and call one tool at a time until the task is complete: 
3. For responding to the email, draft a response email with the write_email tool
4. For meeting requests, use the check_calendar_availability tool to find open time slots
5. To schedule a meeting, use the schedule_meeting tool with a datetime object for the preferred_day parameter
   - Today's date is ${new Date().toISOString().split("T")[0]} - use this for scheduling meetings accurately
6. If you scheduled a meeting, then draft a short response email using the write_email tool
7. After using the write_email tool, the task is complete
8. If you have sent the email, then use the Done tool to indicate that the task is complete
</Instructions>

<Background>
{background}
</Background>

<Response Preferences>
{response_preferences}
</Response Preferences>

<Calendar Preferences>
{cal_preferences}
</Calendar Preferences>
`;
// Create the LLM call node
const llmCallNode = async (state) => {
  /**
   * LLM decides whether to call a tool or not
   * This is the main decision-making node that generates responses or tool calls
   */
  const messages = [...state.messages];
  const systemPromptContent = agentSystemPrompt
    .replace("{tools_prompt}", AGENT_TOOLS_PROMPT)
    .replace("{background}", defaultBackground)
    // .replace("{response_preferences}", defaultResponsePreferences)
    // .replace("{cal_preferences}", defaultCalPreferences);

  // Run the LLM with the messages
  const response = await llmWithTools.invoke([
    { role: "system", content: systemPromptContent },
    ...messages,
  ]);

  // Use explicit casting as the response is compatible with Message in runtime
  return {
    messages: response,
  };
};


#### Tool handler node

After the LLM makes a decision, we need to execute the chosen tool. The `toolHandler` node executes the tool. We can see that nodes can update the graph state to capture any important state changes, such as the classification decision.

In [83]:

async function toolHandler(state) {
  const result = [];
  
  // Get the last message
  const lastMessage = state.messages[state.messages.length - 1];
  
  // Iterate through tool calls
  for (const toolCall of lastMessage.tool_calls || []) {
    // Get the tool
    const tool = toolsByName.get(toolCall.name);
    
    if (!tool) {
      throw new Error(`Tool ${toolCall.name} not found`);
    }
    
    // Run it
    const observation = await tool.invoke(toolCall.args);
    
    // Create a tool message
    result.push({
      role: "tool",
      content: observation,
      tool_call_id: toolCall.id
    });
  }
  
  // Return properly formatted state update
  return { messages: [...state.messages, ...result] };
}

#### Conditional Routing

Our agent needs to decide when to continue using tools and when to stop. This conditional routing function directs the agent to either continue or terminate.

In [84]:
// Define the shouldContinue function needed for conditional edges
const shouldContinue = (state) => {
  const messages = state.messages;
  if (!messages || messages.length === 0) return END;

  const lastMessage = messages[messages.length - 1];
  
  // Check if lastMessage exists and is an AIMessage with tool_calls
  if (
    lastMessage && 
    typeof lastMessage === 'object' &&
    lastMessage.getType && 
    lastMessage.getType() === "ai" &&
    "tool_calls" in lastMessage &&
    Array.isArray(lastMessage.tool_calls) &&
    lastMessage.tool_calls.length > 0
  ) {
    // Check if any tool call is the "Done" tool
    if (lastMessage.tool_calls.some((toolCall) => toolCall.name === "Done")) {
      return END;
    }
    return "interrupt_handler";
  }

  return END;
};

#### Agent Graph

Finally, we can assemble all components:

In [ ]:
import { BaseMessage } from "@langchain/core/messages";
import "@langchain/langgraph/zod";
import { addMessages, Messages, StateGraph, START, END } from "@langchain/langgraph";
// Define the Zod schemas for the email assistant states

const MessagesState = z.object({
  messages: z
    .custom<BaseMessage[]>() // Using any to support all Message types
    .default(() => [])
    .langgraph.reducer<Messages>((left, right) => addMessages(left, right)),
});

 const BaseEmailAgentState = MessagesState.extend({
  email_input: z.any(),
  classification_decision: z
    .enum(["ignore", "respond", "notify"])
    .nullable()
    .default(null),
});

type BaseEmailAgentStateType = z.infer<typeof BaseEmailAgentState>;

// Build workflow
const overallWorkflow = new StateGraph(BaseEmailAgentState)
  .addNode("llm_call", llmCallNode)
  .addNode("tool_handler", toolHandler);

// Add edges
overallWorkflow.addEdge(START, "llm_call");
overallWorkflow.addConditionalEdges(
  "llm_call",
  shouldContinue,
  {
    "tool_handler": "tool_handler",
    [END]: END,
  }
);
overallWorkflow.addEdge("tool_handler", "llm_call");

// Compile the agent
const agent = overallWorkflow.compile();

This creates a graph that:
1. Starts with an LLM decision
2. Conditionally routes to tool execution or termination
3. After tool execution, returns to LLM for the next decision
4. Repeats until completion or no tool is called


### Combine workflow with our agent

We can combine the router and the agent.

In [ ]:
// Combine the triage router and agent workflow
const combinedWorkflow = new StateGraph(StateSchema)
.addNode("triageRouter", triageRouter)
.addNode("response_agent", agent)
.addEdge(START, "triageRouter")
.addConditionalEdges("triageRouter", shouldContinue, {
  "response_agent": "response_agent",
  [END]: END,
})
.addEdge("response_agent", "triageRouter")

// Compile the combined workflow
const finalWorkflow = combinedWorkflow.compile();

This is a higher-level composition where:
1. First, the triage router analyzes the email
2. If needed, the response agent handles crafting a response
3. The workflow ends when either the triage decides no response is needed or the response agent completes

In [ ]:
// Test email input
const testEmailInput = {
  author: "System Admin <sysadmin@company.com>",
  to: "Development Team <dev@company.com>",
  subject: "Scheduled maintenance - database downtime",
  email_thread: "Hi team,\n\nThis is a reminder that we'll be performing scheduled maintenance on the production database tonight from 2AM to 4AM EST. During this time, all database services will be unavailable.\n\nPlease plan your work accordingly and ensure no critical deployments are scheduled during this window.\n\nThanks,\nSystem Admin Team"
};

// Run the agent
const response = await finalWorkflow.invoke({
  email_input: testEmailInput,
  messages: [],
  classification_decision: "notify"  
});

🔔 Classification: NOTIFY - This email contains important information


In [88]:
// Second test email input
const testEmailInput2 = {
  author: "Alice Smith <alice.smith@company.com>",
  to: "John Doe <john.doe@company.com>",
  subject: "Quick question about API documentation",
  email_thread: "Hi John,\nI was reviewing the API documentation for the new authentication service and noticed a few endpoints seem to be missing from the specs. Could you help clarify if this was intentional or if we should update the docs?\nSpecifically, I'm looking at:\n- /auth/refresh\n- /auth/validate\nThanks!\nAlice"
};

// Run the agent with the second test input
const response2 = await finalWorkflow.invoke({
  email_input: testEmailInput2,
  messages: [],
  classification_decision: "respond"  
});



🚫 Classification: IGNORE - This email can be safely ignored


## Testing with Local Deployment

You can find the file for our agent in the `src` directory:

* `src/email_assistant.ts`

You can test them locally in LangGraph Studio by running:

```bash
pnpm agent
```

Example e-mail you can test:

In [89]:
const emailInputExample = {
  author: "Alice Smith <alice.smith@company.com>",
  to: "John Doe <john.doe@company.com>",
  subject: "Quick question about API documentation",
  email_thread: "Hi John,\nI was reviewing the API documentation for the new authentication service and noticed a few endpoints seem to be missing from the specs. Could you help clarify if this was intentional or if we should update the docs?\nSpecifically, I'm looking at:\n- /auth/refresh\n- /auth/validate\nThanks!\nAlice"
};

![studio-img](img/studio.png)